## Vamos a usar as seguintes bibliotecas

In [1]:
#Load required libraries
import numpy as np
import pandas as pd
#For displaying complete rows info
pd.options.display.max_colwidth=500
import tensorflow as tf
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
print(tf.__version__)


2.6.2


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#import os
#workdir_path = '/content/drive/My Drive' #MODIFICAR!!! #Caminho para o seu workspace
#os.chdir(workdir_path)

### Carregamos os artigos

In [4]:
#Load data into pandas dataframe
df=pd.read_csv("folha.csv",encoding="utf8")

In [5]:
df.head(2)

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral","Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está ""lascado"", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer. Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, d...",2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/1925743-lula-diz-que-esta-lascado-mas-que-ainda-tem-forca-como-cabo-eleitoral.shtml
1,"'Decidi ser escrava das mulheres que sofrem', diz cantora Oumou Sangaré","Para Oumou Sangaré, cantora e ativista malinesa, o sofrimento da mulher é universal. ""Canto com todas as minhas forças e vejo mulheres chorando ao meu redor"", disse durante um encontro na Casa da Cultura de Paraty, no Rio de Janeiro, neste domingo (8). Herdeira da tradição musical de sua família –sua avó foi uma ""grande estrela"" do país africano–, ela conta que começou a cantar na rua para ajudar nas finanças de casa após o divórcio dos pais, que deixou seis filhos dependentes de sua mãe, t...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10/1925745-decidi-ser-escrava-das-mulheres-que-sofrem-diz-cantora-oumou-sangare.shtml


In [6]:
print(df["title"][0],"\n",df["text"][0])

Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral 
 Com a possibilidade de uma condenação impedir sua candidatura em 2018, o ex-presidente Luiz Inácio Lula da Silva fez, nesta segunda (9), um discurso inflamado contra a Lava Jato, no qual disse saber que está "lascado", exigiu um pedido de desculpas do juiz Sergio Moro e afirmou que, mesmo fora da disputa pelo Planalto, será um cabo eleitoral expressivo para a sucessão de Michel Temer.  Segundo o petista, réu em sete ações penais, o objetivo de Moro é impedir sua candidatura no ano que vem, desidratando-o, inclusive, no apoio a um nome alternativo, como o do ex-prefeito de São Paulo Fernando Haddad (PT), caso ele não possa concorrer à Presidência.  "Eu sei que tô lascado, todo dia tem um processo. Eu não quero nem que Moro me absolva, eu só quero que ele peça desculpas", disse Lula durante um seminário sobre educação em Brasília. "Eles [investigadores] chegam a dizer: 'Ah, se o Lula não for candidato, ele não vai

## removemos nans

In [7]:
df.shape

(167053, 6)

In [8]:
df.drop_duplicates(subset=["text"],inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

In [9]:
df.shape

(29427, 6)

### WORD2VEC MODEL Usando Gensim

In [10]:
import gensim #usar a versão 3.6.0
import string
import re

In [11]:
print(gensim.__version__)

3.6.0


In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/LeonardoLins/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Deixamos todo mundo minúscula e tiramos a pontuação 

In [13]:
import re
import string

alphanumeric = lambda x: re.sub(r"""\w*\d\w*""", ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) #tira a pontuação nas strings

df['text'] = df.text.map(alphanumeric).map(punc_lower)
df.head()

,title,text,date,category,subcategory,link
0,\n\t\tDa escola ao 1º emprego: como buscar um caminho profissional sem ter (ao menos ainda) um diploma universitário\n\t\t,da bbc brasil o trimestre de maio a julho deste ano foi encerrado com um índice de desemprego na casa de o percentual assusta mas é menor do que os apurados no fim do período imediatamente anterior os dados mostram um quadro preocupante mas que sugere que o pior da crise pode já ter ficado para trás só que a situação dos trabalhadores entre e anos é mais complicada do que a da média segundo a mesma pesquisa do ibge nada menos do que dos jovens nessa faixa etária...,2017-09-10,sobretudo,carreiras,http://www1.folha.uol.com.br/sobretudo/carreiras/2017/10/1925565-da-escola-ao-1-emprego-como-buscar-um-caminho-profissional-sem-ter-ao-menos-ainda-um-diploma-universitario.shtml
1,"50 anos depois da morte, o Che é apenas uma foto na camiseta",ernesto rafael guevara de la serna o homem morreu assassinado na bolívia há exatos anos neste de outubro mas o che o revolucionário morrera antes na cuba em que estão enterrados seus restos o che perdeu internamente a batalha de ideias pelo modelo econômico que deveria ser implantado na ilha prevaleceu o modelo soviético imposto pela necessidade de preservar o financiamento que a união soviética concedia a cuba escreve por exemplo para el país o historiador cubano r...,2017-09-10,colunas,clovisrossi,http://www1.folha.uol.com.br/colunas/clovisrossi/2017/10/1925562-50-anos-depois-da-morte-o-che-e-apenas-uma-foto-na-camiseta.shtml
2,"Afinal, nosso voto tem preço?",a reforma política aprovada pelo congresso deixa claro o seu objetivo impedir a renovação os mecanismos implementados servirão basicamente para resolver o maior problema dos grandes partidos políticos a falta de dinheiro para financiar as suas campanhas essa demanda desesperada por recursos públicos revela a natureza e a qualidade da nossa representatividade os partidos não conseguem convencer o cidadão dos seus planos para o país que justifique um investimento nas suas propostas ao ...,2017-09-10,colunas,joao-amoedo,http://www1.folha.uol.com.br/colunas/joao-amoedo/2017/10/1925545-afinal-nosso-voto-tem-preco.shtml
3,"Cientista dá psicodélico para minicérebros, e eles gostam",a batalha de jovens neurocientistas para reabilitar drogas psicodélicas dá mais um passo nesta segunda feira desta vez com a ajuda de minicérebros um grupo do rio de janeiro tratou essas esferas de células neurais com dmt e constatou uma série de efeitos benéficos a pesquisa aparece no periódico scientific reports do grupo nature sob o título alterações de curto prazo no proteoma de organoides cerebrais humanos induzidas por meo dmt o composto meo dmt pertence à da famíli...,2017-09-10,colunas,marceloleite,http://www1.folha.uol.com.br/colunas/marceloleite/2017/10/1925096-cientista-da-psicodelico-para-minicerebros-e-eles-gostam.shtml
4,Reestruturação da Sabesp deverá ser levada à Justiça por setor de limpeza,associações de empresas de limpeza urbana e resíduos sólidos estudam entrar com uma ação contra a reestruturação societária da sabesp aprovada em setembro o temor é que a companhia paulista passe a oferecer novos serviços —como coleta ou operação de aterros— e aproveite sua capilaridade entre os municípios para ter uma concorrência desleal possivelmente seria uma adi ação direta de inconstitucionalidade ou uma ação junto ao cade órgão que regula a concorrência afirma wladimir rib...,2017-09-10,colunas,mercadoaberto,http://www1.folha.uol.com.br/colunas/mercadoaberto/2017/10/1925191-reestruturacao-da-sabesp-devera-ser-levada-a-justica-por-setor-de-limpeza.shtml


## Tokenizamos e guardamos todo em um unico repositorio

In [14]:
from nltk.tokenize import word_tokenize

## transformando em uma lista de tokens

#### Aplicamos o modelo word2vec. Por default o modelo vem com uma janela de 5. Troque e compare os resultados com os dados na aula agregue window= ??? e 50 dimensões.

In [15]:
corpus = df.text.apply(lambda x:word_tokenize(x)) #transforma em uma série.

In [16]:
corpus[0:2]

0    [da, bbc, brasil, o, trimestre, de, maio, a, julho, deste, ano, foi, encerrado, com, um, índice, de, desemprego, na, casa, de, o, percentual, assusta, mas, é, menor, do, que, os, apurados, no, fim, do, período, imediatamente, anterior, os, dados, mostram, um, quadro, preocupante, mas, que, sugere, que, o, pior, da, crise, pode, já, ter, ficado, para, trás, só, que, a, situação, dos, trabalhadores, entre, e, anos, é, mais, complicada, do, que, a, da, média, segundo, a, mesma, pesquisa, do, ib...
1    [ernesto, rafael, guevara, de, la, serna, o, homem, morreu, assassinado, na, bolívia, há, exatos, anos, neste, de, outubro, mas, o, che, o, revolucionário, morrera, antes, na, cuba, em, que, estão, enterrados, seus, restos, o, che, perdeu, internamente, a, batalha, de, ideias, pelo, modelo, econômico, que, deveria, ser, implantado, na, ilha, prevaleceu, o, modelo, soviético, imposto, pela, necessidade, de, preservar, o, financiamento, que, a, união, soviética, concedia, a, cuba, escrev

In [18]:
model = gensim.models.Word2Vec(corpus, window=3, size=50) # windw = 5 e size = 100 é o padrão

In [19]:
model.wv['lula'] #vetor com 50 posições, pode ser com qualquer valor.

array([-0.9959187 , -0.5379158 , -1.9433067 ,  1.4137323 ,  1.1232227 ,
       -2.8315544 ,  0.45077288, -0.32147077,  1.0426588 , -0.45245352,
        1.4891325 ,  0.9162619 ,  2.1953912 , -0.7633992 ,  1.3284    ,
       -1.1619995 , -0.09599075, -0.1045399 ,  3.7107959 , -0.11836763,
        0.14112523,  1.435117  , -2.728666  ,  0.8227457 ,  0.9997559 ,
        0.32908815,  1.5778072 ,  1.7104937 ,  0.7228189 ,  1.789087  ,
        0.32811582, -0.04522691, -0.12380817,  1.4754689 , -2.1369207 ,
       -1.3448712 ,  0.07504366, -0.45915774, -0.9841062 , -0.7100773 ,
       -1.2693695 , -1.0712262 , -2.3520799 , -1.7305567 , -0.36129797,
       -1.0480233 ,  2.156126  , -0.594176  , -3.2074003 , -1.6943891 ],
      dtype=float32)

In [20]:
model.wv.most_similar("brasil") #distancia vetorial entre pontos mais similares

[('país', 0.879196047782898),
 ('japão', 0.8050692081451416),
 ('mundo', 0.7652781009674072),
 ('exterior', 0.7614580988883972),
 ('méxico', 0.7302500009536743),
 ('planeta', 0.7159854173660278),
 ('entanto', 0.7149247527122498),
 ('canadá', 0.7075656652450562),
 ('chile', 0.6944090723991394),
 ('ocidente', 0.6859036684036255)]

In [21]:
model.wv.most_similar("propina")

[('suborno', 0.7522992491722107),
 ('propinas', 0.7419610619544983),
 ('pensão', 0.701615571975708),
 ('indenização', 0.6947742700576782),
 ('multa', 0.6941344738006592),
 ('empreiteira', 0.6769105195999146),
 ('joesley', 0.665289580821991),
 ('odebrecht', 0.6643797159194946),
 ('petrobras', 0.6596243381500244),
 ('cobrança', 0.6508727669715881)]

In [22]:
model.wv.most_similar("vinhos")

[('doces', 0.798957109451294),
 ('ingredientes', 0.7856484055519104),
 ('sabores', 0.7832441926002502),
 ('pratos', 0.7761790156364441),
 ('tintos', 0.7716960906982422),
 ('queijos', 0.7683677673339844),
 ('peixes', 0.7483096122741699),
 ('menus', 0.7476807832717896),
 ('aromas', 0.7291281819343567),
 ('pães', 0.7286738753318787)]

In [23]:
model.wv.most_similar("petrolão")

[('mensalão', 0.8716286420822144),
 ('escândalo', 0.7617477178573608),
 ('stf', 0.71650230884552),
 ('supremo', 0.705978512763977),
 ('impeachment', 0.7013094425201416),
 ('petismo', 0.69987952709198),
 ('tcu', 0.6979325413703918),
 ('fbi', 0.692789614200592),
 ('judiciário', 0.6678191423416138),
 ('colegiado', 0.6638386249542236)]

In [24]:
model.wv.most_similar("azul")

[('verde', 0.8426753878593445),
 ('marrom', 0.788842499256134),
 ('roxo', 0.7859325408935547),
 ('vermelha', 0.7761906981468201),
 ('amarelo', 0.7749345898628235),
 ('coco', 0.7685152292251587),
 ('gelado', 0.7632014751434326),
 ('cinza', 0.7473464012145996),
 ('amarela', 0.7468852996826172),
 ('frita', 0.7459235787391663)]

In [25]:
model.wv.most_similar("lula")

[('fhc', 0.8616104125976562),
 ('petista', 0.8564023971557617),
 ('dilma', 0.8120459318161011),
 ('pt', 0.7739446759223938),
 ('collor', 0.7662513852119446),
 ('peemedebista', 0.7614772319793701),
 ('temer', 0.7543905973434448),
 ('rousseff', 0.7509742379188538),
 ('levy', 0.7295809984207153),
 ('aécio', 0.7206740975379944)]